# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [2]:
# YOUR CHANGES HERE
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
tickers = ["AAPL", "SPY", "TSLA", "MSFT", "NVDA", "GOOGL"]
start_date = "2024-01-01"
end_date   = "2025-12-31"   # change if Blackboard release was earlier
historical_prices = None

for t in tickers:
    ticker = yf.Ticker(t)
    history = yf.Ticker(t).history(
        start=start_date,
        end=end_date,
        interval="1mo"
    )
    monthly_prices = history['Close'].resample('ME').last()

    if historical_prices is None:
        historical_prices = monthly_prices
    else:
        historical_prices = pd.concat([historical_prices, monthly_prices], axis=1)

historical_prices.columns = tickers
historical_prices.index = pd.to_datetime(historical_prices.index).strftime("%Y-%m")

historical_prices





/usr/local/python/3.12.1/lib/python3.12/site-packages/yfinance/scrapers/history.py:201: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/usr/local/python/3.12.1/lib/python3.12/site-packages/yfinance/scrapers/history.py:201: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/usr/local/python/3.12.1/lib/python3.12/site-packages/yfinance/scrapers/history.py:201: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/usr/local/python/3.12.1/lib/python3.12/site-packages/yfinance/scrapers/history.py:201: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/usr/local/python/3.12.1/lib

,AAPL,SPY,TSLA,MSFT,NVDA,GOOGL
Date,,,,,,
2024-01,182.674820,471.230652,187.289993,391.739227,61.492744,139.049393
2024-02,179.058975,495.822693,201.880005,407.563293,79.067947,137.421692
2024-03,170.092468,510.451111,175.789993,415.305878,90.305687,149.798172
2024-04,168.951782,491.391937,183.279999,384.319794,86.357948,161.559280
2024-05,190.694412,516.246399,178.080002,409.787781,109.577095,171.206421
2024-06,209.199142,532.740906,197.880005,441.994110,123.476997,180.784042
2024-07,220.581818,540.931458,232.070007,413.711243,116.969917,170.449005
2024-08,227.455139,553.570618,214.110001,412.514679,119.318932,162.340927
2024-09,231.696045,563.469910,261.630005,426.295685,121.388046,164.795197


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.



In [3]:
# YOUR CHANGES HERE

# Index = last trading day of each month
# Columns = 6 stock tickers
from pandas.tseries.offsets import BMonthEnd
df = historical_prices.copy()
# Ensure index is datetime
df.index = pd.to_datetime(df.index)
# Convert index to LAST TRADING DAY of each month
df.index = df.index + BMonthEnd(0)
# Create required 'date' column
df = df.reset_index()
df = df.rename(columns={"index": "date"})
# Format date as YYYY-MM-DD
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")
# Save as TSV with required headers
df.to_csv(
    "historical_prices.tsv",
    sep="\t",
    index=False
)
df

,Date,AAPL,SPY,TSLA,MSFT,NVDA,GOOGL
0,2024-01-31,182.674820,471.230652,187.289993,391.739227,61.492744,139.049393
1,2024-02-29,179.058975,495.822693,201.880005,407.563293,79.067947,137.421692
2,2024-03-29,170.092468,510.451111,175.789993,415.305878,90.305687,149.798172
3,2024-04-30,168.951782,491.391937,183.279999,384.319794,86.357948,161.559280
4,2024-05-31,190.694412,516.246399,178.080002,409.787781,109.577095,171.206421
5,2024-06-28,209.199142,532.740906,197.880005,441.994110,123.476997,180.784042
6,2024-07-31,220.581818,540.931458,232.070007,413.711243,116.969917,170.449005
7,2024-08-30,227.455139,553.570618,214.110001,412.514679,119.318932,162.340927
8,2024-09-30,231.696045,563.469910,261.630005,426.295685,121.388046,164.795197
9,2024-10-31,224.645721,560.154602,249.850006,402.568604,132.714554,170.247345


Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [4]:
# YOUR CHANGES HERE
historical_returns = historical_prices.pct_change().dropna()
estimated_returns = historical_returns.mean().rename("return")

Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [5]:
# YOUR CHANGES HERE
# Calculate monthly relative returns
historical_returns = historical_prices.pct_change().dropna()
# Ensure index is datetime (last trading day of month)
historical_returns.index = pd.to_datetime(historical_returns.index)
# Reset index to create 'date' column
historical_returns = historical_returns.reset_index()
historical_returns = historical_returns.rename(columns={"index": "Date"})
# Format date as YYYY-MM-DD
historical_returns["Date"] = historical_returns["Date"].dt.strftime("%Y-%m-%d")
# Save as TSV
historical_returns.to_csv(
    "historical_returns.tsv",
    sep="\t",
    index=False
)
historical_returns.head()

,Date,AAPL,SPY,TSLA,MSFT,NVDA,GOOGL
0,2024-02-01,-0.019794,0.052187,0.077901,0.040394,0.285809,-0.011706
1,2024-03-01,-0.050076,0.029503,-0.129235,0.018997,0.142128,0.090062
2,2024-04-01,-0.006706,-0.037338,0.042608,-0.074610,-0.043715,0.078513
3,2024-05-01,0.128691,0.050580,-0.028372,0.066268,0.268871,0.059713
4,2024-06-01,0.097039,0.031951,0.111186,0.078593,0.126850,0.055942


Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [6]:
expected_returns = estimated_returns.sort_values()

Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [7]:
# YOUR CHANGES HERE

estimated_returns_df = expected_returns.reset_index()
estimated_returns_df.columns = ["asset", "estimated_return"]

# Save as TSV
estimated_returns_df.to_csv(
    "estimated_returns.tsv",
    sep="\t",
    index=False
)

estimated_returns_df

,asset,estimated_return
0,MSFT,0.010854
1,SPY,0.016532
2,AAPL,0.019061
3,GOOGL,0.038990
4,TSLA,0.049640
5,NVDA,0.055531


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [8]:
# YOUR CHANGES HERE
#historical_returns
historical_returns = pd.read_csv("historical_returns.tsv", sep="\t")
historical_returns["Date"] = pd.to_datetime(historical_returns["Date"])
historical_returns.set_index("Date", inplace=True)
covariance_matrix = historical_returns.cov()
covariance_matrix

...

Ellipsis

Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [9]:
# YOUR CHANGES HERE

covariance_matrix.to_csv(
    "estimated_covariance.tsv",
    sep="\t"
)

Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [10]:
# YOUR CHANGES HERE

mu = expected_returns.values             
assets = expected_returns.index.tolist()  # asset names in same order
n = len(mu)
w = cp.Variable(n)

objective = cp.Maximize(mu @ w)
constraints = [
    cp.sum(w) == 1,
    w >= 0
]

problem = cp.Problem(objective, constraints)
problem.solve()

max_return_weights = pd.Series(w.value, index=assets, name="weight")
max_return_weights
Sigma = covariance_matrix.values
portfolio_expected_return = float(mu @ max_return_weights.values)
portfolio_variance = float(max_return_weights.values.T @ Sigma @ max_return_weights.values)
portfolio_volatility = float(np.sqrt(portfolio_variance))
portfolio_expected_return, portfolio_volatility
max_return_weights.sort_values(ascending=False)


"""
The maximum-return portfolio allocates 100% of capital to NVDA, the asset with the highest estimated mean return. 
This concentrated allocation is expected under long-only, fully invested constraints, since diversification does not improve expected return 
when risk is not considered. This result motivates subsequent analysis of portfolio risk and robustness.
"""


'\nThe maximum-return portfolio allocates 100% of capital to NVDA, the asset with the highest estimated mean return. \nThis concentrated allocation is expected under long-only, fully invested constraints, since diversification does not improve expected return \nwhen risk is not considered. This result motivates subsequent analysis of portfolio risk and robustness.\n'

Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [11]:
# YOUR CHANGES HERE

maximum_return_df = max_return_weights.reset_index()
maximum_return_df.columns = ["asset", "allocation"]

# (Optional but recommended) enforce numerical cleanliness
maximum_return_df["allocation"] = maximum_return_df["allocation"].round(6)

# Sanity check: allocations sum to 1
print("Allocation sum:", maximum_return_df["allocation"].sum())

# Save as TSV
maximum_return_df.to_csv(
    "maximum_return.tsv",
    sep="\t",
    index=False
)

maximum_return_df

Allocation sum: 1.0


,asset,allocation
0,MSFT,0.0
1,SPY,0.0
2,AAPL,0.0
3,GOOGL,0.0
4,TSLA,0.0
5,NVDA,1.0


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum risk portfolio based on your previously estimated risks.

In [12]:
# YOUR CHANGES HERE
Sigma = covariance_matrix.values
assets = covariance_matrix.columns.tolist()
n = len(assets)

# Decision variables
w = cp.Variable(n)

# Objective: minimize portfolio variance
objective = cp.Minimize(cp.quad_form(w, Sigma))

# Constraints
constraints = [
    cp.sum(w) == 1,
    w >= 0
]

# Solve
problem = cp.Problem(objective, constraints)
problem.solve()

# Results
min_risk_weights = pd.Series(w.value, index=assets, name="allocation")
min_risk_weights
portfolio_variance = float(min_risk_weights.values.T @ Sigma @ min_risk_weights.values)
portfolio_volatility = float(np.sqrt(portfolio_variance))

portfolio_variance, portfolio_volatility

(0.0009091517640515617, 0.030152143606244013)

Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [13]:
# YOUR CHANGES HERE

# Convert Series to DataFrame with required column names
minimum_risk_df = min_risk_weights.reset_index()
minimum_risk_df.columns = ["asset", "allocation"]

# (Optional) round for clean output
minimum_risk_df["allocation"] = minimum_risk_df["allocation"].round(6)

# Sanity check: allocations sum to 1
print("Allocation sum:", minimum_risk_df["allocation"].sum())

# Save as TSV
minimum_risk_df.to_csv(
    "minimum_risk.tsv",
    sep="\t",
    index=False
)

minimum_risk_df

Allocation sum: 0.9999990000000001


,asset,allocation
0,AAPL,0.114909
1,SPY,0.844631
2,TSLA,-0.000000
3,MSFT,-0.000000
4,NVDA,-0.000000
5,GOOGL,0.040459


Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [14]:
# YOUR CHANGES HERE

# Ensure consistent asset order everywhere
assets = expected_returns.index.tolist()
mu = expected_returns.loc[assets].values
Sigma = covariance_matrix.loc[assets, assets].values

w_min = min_risk_weights.loc[assets].values
w_max = max_return_weights.loc[assets].values

# Compute endpoint portfolio returns
min_risk_return = float(mu @ w_min)
max_return = float(mu @ w_max)

# 101 evenly spaced target returns
target_returns = np.linspace(min_risk_return, max_return, 101)

weights_list = []

for i, R_target in enumerate(target_returns):
    # Force endpoints to match previous parts exactly
    if i == 0:
        weights_list.append(w_min)
        continue
    if i == len(target_returns) - 1:
        weights_list.append(w_max)
        continue

    # Decision variable
    w = cp.Variable(len(assets))

    # Minimize variance subject to hitting target return
    objective = cp.Minimize(cp.quad_form(w, Sigma))
    constraints = [
        cp.sum(w) == 1,
        w >= 0,
        mu @ w == R_target
    ]

    prob = cp.Problem(objective, constraints)

    # Solve (try a couple solvers for robustness)
    solved = False
    for solver in [cp.OSQP, cp.SCS]:
        try:
            prob.solve(solver=solver, verbose=False)
            if w.value is not None:
                weights_list.append(w.value)
                solved = True
                break
        except Exception:
            pass

    if not solved:
        raise RuntimeError(f"Optimization failed at i={i}, target return={R_target:.6f}")

# Assemble into a DataFrame (rows = portfolios, cols = assets)
frontier_weights = pd.DataFrame(weights_list, columns=assets)
frontier_weights.index = pd.Index(range(101), name="portfolio_id")

# Compute frontier portfolio returns & risks for sanity checks / plotting
frontier_returns = frontier_weights.values @ mu
frontier_vars = np.einsum("ij,jk,ik->i", frontier_weights.values, Sigma, frontier_weights.values)
frontier_vols = np.sqrt(frontier_vars)

frontier_summary = pd.DataFrame({
    "target_return": target_returns,
    "portfolio_return": frontier_returns,
    "portfolio_volatility": frontier_vols
})

frontier_summary.head(), frontier_summary.tail()


(   target_return  portfolio_return  portfolio_volatility
 0       0.017731          0.017731              0.030152
 1       0.018109          0.018109              0.030181
 2       0.018487          0.018487              0.030269
 3       0.018865          0.018865              0.030414
 4       0.019243          0.019243              0.030616,
      target_return  portfolio_return  portfolio_volatility
 96        0.054019          0.054019              0.097261
 97        0.054397          0.054397              0.100336
 98        0.054775          0.054775              0.104727
 99        0.055153          0.055153              0.110278
 100       0.055531          0.055531              0.116823)

Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [18]:
# YOUR CHANGES HERE
import pandas as pd

# Build final DataFrame
efficient_frontier_df = frontier_weights.copy()
efficient_frontier_df.insert(0, "risk", frontier_vols)
efficient_frontier_df.insert(0, "return", frontier_returns)
efficient_frontier_df.insert(0, "index", range(len(efficient_frontier_df)))

# Sanity checks
print("Rows:", len(efficient_frontier_df))          # should be 101
print("Allocation sum (max deviation):",
      (efficient_frontier_df[assets].sum(axis=1) - 1).abs().max())

# Save as TSV
efficient_frontier_df.to_csv(
    "efficient_frontier.tsv",
    sep="\t",
    index=False
)

efficient_frontier_df.head(), efficient_frontier_df.tail()


Rows: 101
Allocation sum (max deviation): 2.220446049250313e-16


(              index    return      risk          MSFT       SPY      AAPL  \
 portfolio_id                                                                
 0                 0  0.017731  0.030152 -5.614903e-20  0.844631  0.114909   
 1                 1  0.018109  0.030181 -4.093410e-20  0.829765  0.112695   
 2                 2  0.018487  0.030269 -4.034592e-20  0.814899  0.110481   
 3                 3  0.018865  0.030414 -3.975911e-20  0.800033  0.108267   
 4                 4  0.019243  0.030616 -3.917114e-20  0.785167  0.106052   
 
                  GOOGL          TSLA          NVDA  
 portfolio_id                                        
 0             0.040459 -3.685562e-21 -6.903915e-20  
 1             0.057540  1.295085e-20 -1.396822e-20  
 2             0.074620  1.276548e-20 -1.376533e-20  
 3             0.091700  1.257729e-20 -1.356888e-20  
 4             0.108781  1.239480e-20 -1.336442e-20  ,
               index    return      risk          MSFT           SPY  \
 

Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [16]:
# YOUR CHANGES HERE

# --- Inputs (from earlier parts) ---
assets = expected_returns.index.tolist()
mu = expected_returns.values                  
Sigma = covariance_matrix.loc[assets, assets].values  # ensure same order, shape (6,6)

# --- Resampling parameters ---
n_months = 23
n_trials = 1000
rng = np.random.default_rng(42)  # reproducible

winners = []

for _ in range(n_trials):
    # 1) Simulate 23 monthly returns from multivariate normal
    simulated = rng.multivariate_normal(mean=mu, cov=Sigma, size=n_months)  # shape (23,6)

    # 2) Re-estimate expected returns from the simulated history
    simulated_mean = simulated.mean(axis=0)  # shape (6,)

    # 3) Identify which asset has the highest re-estimated mean return
    winner_idx = int(np.argmax(simulated_mean))
    winners.append(assets[winner_idx])

# Count how often each asset "wins"
winner_counts = pd.Series(winners).value_counts().reindex(assets, fill_value=0)
winner_freq = (winner_counts / n_trials).rename("win_frequency")

results = pd.concat([winner_counts.rename("win_count"), winner_freq], axis=1)
results.sort_values("win_frequency", ascending=False)

top_asset = results["win_frequency"].idxmax()
top_freq = results.loc[top_asset, "win_frequency"]

print(f"Most frequent max-return asset: {top_asset}")
print(f"Win frequency over {n_trials} resamples: {top_freq:.3f}")



Most frequent max-return asset: NVDA
Win frequency over 1000 resamples: 0.467


Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [17]:
# YOUR CHANGES HERE
# Count wins for each asset
win_counts = pd.Series(winners).value_counts().reindex(assets, fill_value=0)

# Convert to probabilities
probabilities = (win_counts / n_trials).rename("probability")

# Create final DataFrame
max_return_prob_df = probabilities.reset_index()
max_return_prob_df.columns = ["asset", "probability"]

# Sanity check: probabilities sum to 1
print("Probability sum:", max_return_prob_df["probability"].sum())

# Save as TSV
max_return_prob_df.to_csv(
    "max_return_probabilities.tsv",
    sep="\t",
    index=False
)

max_return_prob_df


Probability sum: 1.0


,asset,probability
0,MSFT,0.000
1,SPY,0.000
2,AAPL,0.007
3,GOOGL,0.124
4,TSLA,0.402
5,NVDA,0.467


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.